Count pixels that are reclassified from CONV to STRA and are classified as certain shallow rain.

In [ ]:
import os
import netCDF4 as nc4
import numpy as np
import numpy.ma as ma

In [ ]:
indir = '/home/disk/bob/gpm/nam_ku/classify/ex_data/2015/06'
outfile = indir+'/shallowRainCheck.txt'

In [ ]:
fout = open(outfile, 'w')
totRainPix = 0
totRainPixChg = 0

In [ ]:
for fname in os.listdir(indir):
    if fname.endswith('nc'):
        print fname
        # get data of interest
        ncid = nc4.Dataset(indir+'/'+fname,'r')
        rt = ncid.variables['rain_type'][:]
        rt_missing = ncid.variables['rain_type'].missing_value
        rt_uw = ncid.variables['rain_type_uw'][:]
        srt = ncid.variables['shallow_rain_type'][:]
        ncid.close()
        
        fltr = np.array(rt,copy=True)
        fltr[:,:,:] = 0

        # get total 2D pixels
        totRainPix = totRainPix + (rt.size - np.count_nonzero(rt.data == rt_missing))
        
        #count pixels where:
        # 1. rt==2
        # 2. rt_uw==1
        # 3. srt==11 or 21
        chgRT = np.logical_and(rt==2,rt_uw==1)
        chgSRT = np.logical_or(srt==11,srt==21)
        chgTOT = np.logical_and(chgRT,chgSRT)
        fltr[chgTOT == True] = 1
        if np.sum(fltr) > 0:
            totRainPixChg = totRainPixChg + np.sum(fltr)
            fout.write("fname = {} and RainPixChg = {}\n".format(fname,np.sum(fltr)))
        

In [ ]:
fout.write("totRainPixChg/totRainPix = {}/{} = {}%\n".format(totRainPixChg,totRainPix,totRainPixChg/totRainPix))
fout.close()